# Laboratorio 8 - Word Embeddings
## <font size=4> *CC3092 - Deep Learning y Sistemas Inteligentes*, 2024 </font>
## <font size=3 color='gray'> Samuel Chamale y Adrian Rodriguez</font>

### Generar Embeddings con Word2Vec

In [25]:
# Importar las librerías necesarias
import gensim
from gensim.models import Word2Vec
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import string
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [26]:
# Descargar recursos de NLTK (si no se han descargado previamente)
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chama\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
# Cargar los datos desde el archivo de texto
# Se intentó con 'utf-8' pero no funcionó
# al parecer el archivo está codificado en 'latin1', fue coincidencia haberlo descubierto,
# probé con varios y este fue el único que funcionó (e.g. 'utf-8', 'utf-16', 'utf-32', 'ascii', 'cp1252')
with open('reviews_data.txt', 'r', encoding='latin1') as file:
    documents = file.readlines()

In [28]:
# Lista para almacenar solo las reseñas
reviews = []
malformed_lines = 0

for line in documents:
    sections = line.strip().split('\t')
    if len(sections) == 3:
        date, title, review = sections
        reviews.append(review)
    else:
        malformed_lines += 1

print(f"Se encontraron {malformed_lines} líneas mal formadas (esto significa un porcentaje de: {malformed_lines/len(documents)*100:.2f}%)")

Se encontraron 28957 líneas mal formadas (esto significa un porcentaje de: 11.30%)


In [29]:
# Preprocesar los documentos AKA reseñas
def preprocess(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar puntuación
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenizar
    tokens = word_tokenize(text)
    return tokens

In [30]:
# Aplicar preprocesamiento a todos los documentos
processed_docs = [preprocess(review) for review in reviews]

In [31]:
model = Word2Vec(
    sentences=processed_docs,
    vector_size=100,  # Tamaño del vector de embeddings
    window=5,         # Tamaño de la ventana de contexto
    min_count=5,      # Ignorar palabras con frecuencia menor a 5
    workers=4,        # Número de hilos para el entrenamiento
    sg=0              # Uso de CBOW (sg=1 para Skip-gram)
)

In [32]:
model.train(processed_docs, total_examples=len(processed_docs), epochs=10)

(294622642, 415389570)

In [33]:
# Guardar el modelo entrenado
# model.save("word2vec_review_data.model")

In [34]:
# Cargar el modelo entrenado
# model = Word2Vec.load("word2vec_review_data.model")

### Encontrar las 10 Palabras Más Similares

In [35]:
# Lista de palabras objetivo
target_words = ['street', 'good', 'dog', 'mother', 'bed']

# Diccionario para almacenar las palabras similares
similar_words = {}

for word in target_words:
    if word in model.wv:
        similar = model.wv.most_similar(word, topn=10)
        similar_words[word] = [w for w, score in similar]
        print(f"Palabras similares a '{word}': {similar_words[word]}")
    else:
        print(f"La palabra '{word}' no se encuentra en el vocabulario.")
        similar_words[word] = []

Palabras similares a 'street': ['road', 'streetthe', 'rd', 'steet', 'streetit', 'streeti', 'streetits', 'st', 'stree', 'streeta']
Palabras similares a 'good': ['decent', 'great', 'terrific', 'goodthe', 'nice', 'excellent', 'fantastic', 'fab', 'superb', 'excellentthe']
Palabras similares a 'dog': ['dogs', 'child', 'baby', 'pup', 'cat', 'pet', 'companion', 'doll', 'toddler', 'swimsuit']
Palabras similares a 'mother': ['father', 'niece', 'mom', 'mum', 'sisterinlaw', 'cousin', 'daughter', 'neice', 'brother', 'nephew']
Palabras similares a 'bed': ['beds', 'bedthe', 'sofabed', 'mattress', 'daybed', 'bed2', 'bedsour', 'hideabed', 'bedwe', 'bedsclean']
